# 鳥の群れ
houdiniでpython SOPノードで実装を予定

参考  
https://qiita.com/piyopiyomaru/items/8c2bf12356303b28fc32


### Boidの基本的アルゴリズム  
- Collision Avoidance: 群れの仲間との衝突を避ける  
- Velocity Matching: 群れの仲間と速度を合わせようとする  
- Flock Centering: 群れの仲間と近づいたままでいようとする  
https://en.m.wikipedia.org/wiki/Boids  
(http://www.cs.toronto.edu/~dt/siggraph97-course/cwr87/)
["Boids Background and Update" by Craig Reynolds]


## 座標と進行方向ベクトル

In [3]:
import numpy as np

width = 25 # フィールド正方形の１辺の長さ。つまりx座標とy座標のとりうる範囲

num_of_birds = 100 # 鳥の数。とりあえず100匹にしておく

pos = np.random.rand(num_of_birds, 2) * (0.6*width - 0.4*width) + 0.4*width # 初期座標をランダム生成する
vec = np.random.rand(num_of_birds, 2) - 0.5 # 初期ベクトルをランダム生成する


## 1. 距離行列を作る

In [7]:
def DISTANCE() :

    global distance # 距離行列の表であるdistanceはグローバルなオブジェクトとして定義しておく
    
    distance = np.zeros([num_of_birds, num_of_birds]) # 空のndarray配列を(鳥の数)*(鳥の数)で作っておく

    for i in range (num_of_birds) :

        for j in range (i+1, num_of_birds) :

            a = pos[i] # 表の横列
            b = pos[j] # 表の縦列

            dist = np.linalg.norm(b-a) # 座標間の距離を計算 #Houdiniでのpointの3次元距離計算にも対応

            distance[i, j] = dist # 距離行列の表のマスに代入

    distance += distance.T # 表の上半分を反転させて、下半分のマスを埋める
